# Openrave for grasping

First we load the robot, a scene and a mug to grasp it.

In [1]:
from openravepy import *
import numpy
env = Environment() # create openrave environment
env.SetViewer('qtcoin') # attach viewer (optional)
RaveSetDebugLevel(DebugLevel.Debug)


env.Load("../../fh_desc/model.xml")
env.Load("data/table_washer.kinbody.xml")

robot = env.GetRobots()[0]
manip = robot.GetActiveManipulator()

ikmodel = databases.inversekinematics.InverseKinematicsModel(robot,iktype=IkParameterization.Type.Transform6D)
if not ikmodel.load():
    ikmodel.autogenerate()

env.Load("data/mug1.kinbody.xml")

mug = env.GetKinBody('mug')
mug_pose = numpy.array([[1,0,0,0],[0,0,-1,0],[0,1,0,0.05],[0,0,0,1]])

mug.SetTransform(mug_pose)

manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs


Now we load or generate (if necessary) the list of grasps

In [2]:
gmodel = databases.grasping.GraspingModel(robot,mug)
if not gmodel.load():
    gmodel.autogenerate()

Now we use the first valid grasp

In [3]:
validgrasps, validindicees = gmodel.computeValidGrasps(returnnum=1)
gmodel.moveToPreshape(validgrasps[0])
Tgoal = gmodel.getGlobalGraspTransform(validgrasps[0],collisionfree=True)
basemanip = interfaces.BaseManipulation(robot)
basemanip.MoveToHandPosition(matrices=[Tgoal])
robot.WaitForController(0)
taskmanip = interfaces.TaskManipulation(robot)
taskmanip.CloseFingers()
robot.WaitForController(0)

True

# Other useful openrave tricks

Plans in cartesian

In [4]:
for _ in range(1):
    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.37,0.92,0,0.4],[0,0,1,-0.23],[0.93,-0.37,0,0.12],[0,0,0,1]])
    res = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10) # call motion planner with goal joint angles
    robot.WaitForController(0) # wait

    manipprob = interfaces.BaseManipulation(robot) # create the interface for basic manipulation programs
    Tgoal = numpy.array([[0.51,0.86,0,-0.4],[0,0,1,-0.23],[0.86,-0.51,0,0.12],[0,0,0,1]])
    res = manipprob.MoveToHandPosition(matrices=[Tgoal],seedik=10) # call motion planner with goal joint angles
    robot.WaitForController(0) # wait